Some notes
Initial dimensions:23x11056

### DATASET COLUMNS
- **PLAYER NAME:** Name of the player.
- **PLAYER TEAM:** The team to which the player belongs.
- **PLAYER POSITION:** Position played by the player (e.g., quarterback, running back, wide receiver).
- **LOC:** Location of the game (home or away).
- **OPP:** Opponent team.
- **STATUS:** Player's status (e.g., active, injured).
- **PROJ:** Projected performance.
- **PASSING C/A:** Passing completions/attempts.
- **PASSING YDS:** Passing yards.
- **PASSING TD:** Passing touchdowns.
- **PASSING INT:** Passing interceptions.
- **RUSHING CAR:** Rushing attempts.
- **RUSHING YDS:** Rushing yards.
- **RUSHING TD:** Rushing touchdowns.
- **RECEIVING REC:** Receptions.
- **RECEIVING YDS:** Receiving yards.
- **RECEIVING TD:** Receiving touchdowns.
- **RECEIVING TAR:** Receiving targets.
- **MISC 2PC:** Miscellaneous 2-point conversions.
- **MISC FUML:** Miscellaneous fumbles.
- **MISC TD:** Miscellaneous touchdowns.
- **TOTAL:** Overall total or combined score.


### POINT CALCULATION
**Quarterback (QB):**
- **Passing Points:**
  - Passing Points = (Passing Yards / Yardage Points) + (Passing Touchdowns * Touchdown Points) - (Interceptions * Interception Points)

**Running Back (RB):**
- **Rushing Points:**
  - Rushing Points = (Rushing Yards / Rushing Yardage Points) + (Rushing Touchdowns * Touchdown Points)
- **Receiving Points:**
  - Receiving Points = (Receptions * Reception Points) + (Receiving Yards / Receiving Yardage Points) + (Receiving Touchdowns * Touchdown Points)
- **Total Points for RB:**
  - Total Points = Rushing Points + Receiving Points

**Wide Receiver (WR):**
- **Receiving Points:**
  - Receiving Points = (Receptions * Reception Points) + (Receiving Yards / Receiving Yardage Points) + (Receiving Touchdowns * Touchdown Points)

**Tight End (TE):**
- **Receiving Points:**
  - Receiving Points = (Receptions * Reception Points) + (Receiving Yards / Receiving Yardage Points) + (Receiving Touchdowns * Touchdown Points)
